# Online Softmax增量计算 - 实践篇

本notebook通过实际代码帮助你理解Online Softmax算法。

**学习目标：**
- 实现逐元素的Online Softmax
- 实现块级Online Softmax
- 验证Online算法与标准算法的结果一致


## 环境准备


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print(f"NumPy版本: {np.__version__}")


## 1. 标准Softmax（作为对照）


In [ ]:
def standard_softmax(x):
    """标准的数值稳定Softmax"""
    m = np.max(x)
    exp_x = np.exp(x - m)
    l = np.sum(exp_x)
    return exp_x / l

# 测试
x = np.array([2.0, 1.0, 3.0, 0.5, 2.5])
result = standard_softmax(x)
print(f"输入: {x}")
print(f"输出: {result}")
print(f"验证和为1: {np.sum(result):.6f}")


## 2. 逐元素Online Softmax


In [ ]:
def online_softmax_element_by_element(x, verbose=False):
    """
    逐元素的Online Softmax
    
    核心思想：
    - 维护运行时状态 (m, l)
    - m: 当前最大值
    - l: 当前（相对于m的）累加和
    
    更新规则：
    - m_new = max(m, x_i)
    - l_new = l * exp(m - m_new) + exp(x_i - m_new)
    """
    n = len(x)
    
    # 初始化：处理第一个元素
    m = x[0]  # 最大值
    l = 1.0   # exp(x[0] - m) = exp(0) = 1
    
    if verbose:
        print("=" * 60)
        print("逐元素Online Softmax过程")
        print("=" * 60)
        print(f"初始化: x[0]={x[0]:.2f}, m={m:.2f}, l={l:.4f}")
    
    # 遍历剩余元素
    for i in range(1, n):
        m_prev = m
        m = max(m, x[i])  # 更新最大值
        
        # 关键：缩放之前的累加和
        l = l * np.exp(m_prev - m) + np.exp(x[i] - m)
        
        if verbose:
            scale_factor = np.exp(m_prev - m)
            print(f"处理 x[{i}]={x[i]:.2f}:")
            print(f"  m: {m_prev:.2f} → {m:.2f}")
            print(f"  缩放因子: exp({m_prev:.2f}-{m:.2f}) = {scale_factor:.4f}")
            print(f"  l: {l:.4f}")
    
    if verbose:
        print(f"\n最终状态: m={m:.2f}, l={l:.4f}")
    
    # 计算最终结果
    result = np.exp(x - m) / l
    return result, m, l

# 测试
x = np.array([2.0, 1.0, 3.0, 0.5, 2.5])
result_online, m, l = online_softmax_element_by_element(x, verbose=True)

print(f"\n结果: {result_online}")
print(f"验证和为1: {np.sum(result_online):.6f}")


In [ ]:
# 验证Online结果与标准结果一致
print("=" * 50)
print("验证Online Softmax与标准Softmax结果一致")
print("=" * 50)

x = np.array([2.0, 1.0, 3.0, 0.5, 2.5])
result_standard = standard_softmax(x)
result_online, _, _ = online_softmax_element_by_element(x, verbose=False)

print(f"\n标准Softmax: {result_standard}")
print(f"Online Softmax: {result_online}")
print(f"最大差异: {np.max(np.abs(result_standard - result_online)):.2e}")
print("\n✓ 结果完全一致！")


## 3. 块级Online Softmax
